질문
1. train, test 데이터를 5프로만 사용하면 label이 0인 부정적인 데이터만 학습될 것 같은데 맞나요?
2. 왜 제 코드는 잘 학습되지 않을까요? train loss가 그대로예요...

In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

# IMDb 데이터셋 로드
# train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
# test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")
train_ds = load_dataset("stanfordnlp/imdb", split="train")
test_ds = load_dataset("stanfordnlp/imdb", split="test")

# Hugging Face에서 사전 학습된 BERT 토크나이저 로드
# 사전 학습된 BERT 토크나이저란 : bert-base-uncased가 학습될 때 사용했던 동일한 방식의 토크나이저
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


# 배치 단위로 데이터를 전처리하는 함수
def collate_fn(batch):
  max_len = 512  # 입력 시퀀스 최대 길이 설정
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


# train data는 매 epoch마다 데이터 섞음. 그렇지 않으면 항상 같은 순서로 학습해서 순서에 의존 할 수 있음
train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### 입력 시퀀스
* 고정된 길이의 텐서로 모델에 입력하기 위해 설정하는 하이퍼파라미터. 하나의 문장을 "최대 몇 개의 단어(또는 토큰)"로 자를지를 정하는 값
   * 왜 max_len 400인가 : bert-base-uncased 모델의 최대 입력 길이는 512 토큰인데, 512까지 쓰면 성능은 좋아질 수 있지만 메모리 사용량이 커지고 학습 속도는 느려질 수 있음.
   * 512까지 쓰면 메모리 사용량이 커지고 학습 속도가 느려지는 이유 : BERT는 Self-Attention 메커니즘을 사용하는데, 입력 토큰 수가 많을수록 계산량이 기하급수적으로 증가함. Self-Attention은 O(N²)의 시간/메모리 복잡도로 512일 경우 512×512 = 262,144 연산, 400일 경우 160,000 연산

In [ ]:
from torch import nn
from math import sqrt


class MultiHeadAttention(nn.Module):
  def __init__(self, input_dim, d_model, n_heads):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.n_heads = n_heads

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    B, S, D = q.shape[0], q.shape[1], self.d_model // self.n_heads

    q = q.reshape((B, S, self.n_heads, D)).transpose(1, 2)
    k = k.reshape((B, S, self.n_heads, D)).transpose(1, 2)
    v = v.reshape((B, S, self.n_heads, D)).transpose(1, 2)

    score = torch.matmul(q, k.transpose(-1, -2)) # (B, H, S, D) * (B, H, D, S) = (B, H, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask[:, None] * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)  # (B, H, S, D)

    result = result.transpose(1, 2).reshape((B, S, -1))
    result = self.dense(result)

    return result

In [ ]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, n_heads, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.n_heads = n_heads
    self.dff = dff

    self.sa = MultiHeadAttention(input_dim, d_model, n_heads)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

    self.norm1 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(0.1)

    self.norm2 = nn.LayerNorm(d_model)
    self.dropout2 = nn.Dropout(0.1)

  def forward(self, x, mask):
    x1 = self.sa(x, mask)
    x1 = self.dropout1(x1)
    x1 = self.norm1(x + x1)

    x2 = self.ffn(x1)
    x2 = self.dropout2(x2)
    x2 = self.norm2(x1 + x2)

    return x

In [ ]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 512
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 512, 256])


In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_heads, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_heads = n_heads
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, n_heads, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 4, 5, 32)

In [ ]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
n_epochs = 100
best_acc = 0.0  # 최고 test accuracy 저장용

for epoch in range(n_epochs):
    total_loss = 0.
    model.train()

    for data in train_loader:
        model.zero_grad()
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

        preds = model(inputs)[..., 0]
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss:.4f}")

    with torch.no_grad():
        model.eval()
        train_acc = accuracy(model, train_loader)
        test_acc = accuracy(model, test_loader)
        print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

        # === 체크포인트 저장 ===
        if test_acc >= best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'test_acc': test_acc
            }, '/content/drive/MyDrive/Colab Notebooks/best_checkpoint.pt')
            print(f"✅ New best model saved at epoch {epoch} (Test acc: {test_acc:.3f})")


Epoch   0 | Train Loss: 279.4696
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 0 (Test acc: 0.500)
Epoch   1 | Train Loss: 271.5814
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 1 (Test acc: 0.500)
Epoch   2 | Train Loss: 272.1967
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 2 (Test acc: 0.500)
Epoch   3 | Train Loss: 271.4180
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 3 (Test acc: 0.500)
Epoch   4 | Train Loss: 271.9041
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 4 (Test acc: 0.500)
Epoch   5 | Train Loss: 271.9173
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 5 (Test acc: 0.500)
Epoch   6 | Train Loss: 271.8607
=========> Train acc: 0.500 | Test acc: 0.500
✅ New best model saved at epoch 6 (Test acc: 0.500)
Epoch   7 | Train Loss: 271.9915
=========> Train acc: 0.500 | Test acc: 0.500
✅ Ne

KeyboardInterrupt: 

In [ ]:
# # 모델 저장
# model_path = "/content/drive/MyDrive/Colab Notebooks/best_checkpoint.pt"
# torch.save(model.state_dict(), model_path)
# print(f"모델이 {model_path}에 저장되었습니다.")

In [ ]:
def predict(text, model, tokenizer, max_len=512):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, truncation=True, max_length=max_len, return_tensors="pt")
        input_ids = inputs['input_ids'][:, :-1].to('cuda')  # 마지막 토큰 제외
        output = model(input_ids)
        pred = torch.argmax(output, dim=-1).item()
        return pred

In [ ]:
label_map = {
    0: "부정적인 리뷰 😞",
    1: "긍정적인 리뷰 😊"
}

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/best_checkpoint.pt"
loaded_model = TextClassifier(len(tokenizer), 32, 4, 5, 32)
checkpoint = torch.load(model_path)
loaded_model.load_state_dict(checkpoint['model_state_dict'])

loaded_model = loaded_model.to('cuda')
loaded_model.eval()
print(loaded_model)

TextClassifier(
  (embedding): Embedding(30522, 32)
  (layers): ModuleList(
    (0-4): 5 x TransformerLayer(
      (sa): MultiHeadAttention(
        (wq): Linear(in_features=32, out_features=32, bias=True)
        (wk): Linear(in_features=32, out_features=32, bias=True)
        (wv): Linear(in_features=32, out_features=32, bias=True)
        (dense): Linear(in_features=32, out_features=32, bias=True)
        (softmax): Softmax(dim=-1)
      )
      (ffn): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
      )
      (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (classification): Linear(in_features=32, out_features=1, bias=True)
)


In [ ]:
text = """I turned over to this film in the middle of the night and very nearly skipped right passed it. It was only because there was nothing else on that I decided to watch it. In the end, I thought it was great.<br /><br />An interesting storyline, good characters, a clever script and brilliant directing makes this a fine film to sit down and watch. This was, in fact, the first I'd heard of this movie, but I would have been happy to have paid money to see this at the cinema.<br /><br />My IMDB Rating : 8 out of 10<br /><br />"""
pred = predict(text, loaded_model, tokenizer)
print(f"예측 결과: {label_map.get(pred, '알 수 없음')}")

예측 결과: 부정적인 리뷰 😞


In [ ]:
test_ds['text'][20000]

"I turned over to this film in the middle of the night and very nearly skipped right passed it. It was only because there was nothing else on that I decided to watch it. In the end, I thought it was great.<br /><br />An interesting storyline, good characters, a clever script and brilliant directing makes this a fine film to sit down and watch. This was, in fact, the first I'd heard of this movie, but I would have been happy to have paid money to see this at the cinema.<br /><br />My IMDB Rating : 8 out of 10<br /><br />"

In [ ]:
test_ds['label'][20000]

1